
# M-P神经元模型

![image-20201015201324959](assets/image-20201015201324959.png)



## 激活函数

![image-20201015202112694](assets/image-20201015202112694.png)



理想的激活函数为阶跃函数，因为不可导，可以用sigmoid函数。

激活函数要求（这段来机器学习从入门到实战）：

- 非线性：线性组成依然是线性，表达能力有限
- 连续可微（ReLU除外：不连续）,大部分用到梯度下降
- 值域有限：否则出现一个1000000，一个10,训练难以稳定
- 单调性：保证单层神经网络凸函数
- 最好导数也是单调光滑
- 函数值与输入至近似相等，这样初始化参数很小值时训练也会很高效

TODO 常用激活函数：

1） Sigmod: 缺点也明显：导致梯度消失（有效范围太短，值很大很小的时候，变化趋于平缓），输出非0均值。输出都是正值，梯度也是正值，导致锯齿型。 导数涉及除法，训练过程反向传播时计算量特别大

2） Tanh，双曲三角函数$tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$;相比sigmod有改进，但是依然没有解决梯度消失问题，本质上是：$tanh(x) = 2 * sigmod(x) - 1$;最好不在隐含层使用，梯度消失。可以在输出层使用。

3）Hard Tanh:$f(x) =\left\{\begin{matrix}
+1, x > 1\\ 
-1, x < -1\\ 
x, other
\end{matrix}\right.$

4）ReLU：$f(x) = max(0, x)$;模仿生物（对一部分信号忽略，对一部分信号产生反映），也有缺点，会死（训练一段时间后，只会输出0,尤其是学习率很大的时候，也就是一个很大数据流过后，可能变为0,梯度都是负数了），pytorch有一个参数inplace用来选择是否进行覆盖运算

还有很多改进版：Leaky ReLU,小于0时不再输出0：$f(x) =max(\alpha x, x)$，实际并未总是优于ReLU

Parametric ReLU:（上述参数是学习出来的）

ELU：$f(x) =\left\{\begin{matrix}
\alpha(exp(x) - 1), x < 0\\ 
x, other
\end{matrix}\right.$，虽然拥有所有有点，但是实际证明也并未总是好于ReLU，计算还比较慢，因为有指数运算。收敛会快一点。



5） Maxout：会导致参数K倍增长

6）Softmax：http://ufldl.stanford.edu/tutorial/supervised/SoftmaxRegression/，需要加上正则项，不是严格非凸

7）LogSoftmax：增加log


# 感知机

最古老，感知机只有2层，只能实现与，或，非等线性可分的，不能实现典型异或线性不可分问题。



# BP算法

训练集$D=\{(x_1,y_1), (x_2,y_2),..., (x_m,y_m)\},\pmb{x}_i \in \mathbb{R}^d,\pmb{y}_i \in \mathbb{R}^l$，样本d维特征，输出$l$维实值向量，输出层第j个神经元的阈值为$\theta_j$，隐藏层第h个原本的阈值为$\gamma_h$。

假定网络结构如：

![image-20201015210229745](assets/image-20201015210229745.png)

假设隐藏层与输出层都使用sigmod函数，对于训练样本$(\pmb{x}_k, \pmb{y}_k)$，假定神经网络的输出为$(\hat{y}_1^k, \hat{y}_2^k,...,\hat{y}_l^k)$，即
$$
\hat{y}_j^k = f(\beta_j - \theta_j)
$$
则网络在$(\pmb{x}_k, \pmb{y}_k)$的均方误差为：
$$
E_k=\frac{1}{2}\sum_{j=1}^{l}(\hat{y}_j^k - y_j^k)^2
$$

则参数个数：(d + l + 1)q +l，输入层到隐藏层有d * q个权值+q个阈值，输入层到输出层有q*l个权重+l个阈值。

BP是一个迭代学习算法，迭代的没一轮采用广义的感知机学习规则对参数进行更新估计，即:
$$
{v \leftarrow v + \Delta v\\
\Delta v = \eta (y - \hat{y}_i)x_i
}
$$
以$w_{hj}$为例推导，BP基于梯度下降策略，以目标负梯度方向对参数进行调整（损失函数选择平方误差，学习率$\eta$，梯度下降与泰勒公式关系参考[梯度下降法和一阶泰勒展开的关系](https://zhuanlan.zhihu.com/p/82757193)）
$$
\Delta w_{hj} = -\eta \frac{\partial E_k}{\partial w_{hj}}
$$
注意到 $w_{hj}$ 先影响到第 j 个输出层神经元的输入值$\beta_j$,再影响到其输出值$\hat{y}_j^k$,
然后影响到 $E_k$ , 有
$$
\frac{\partial E_k}{\partial w_{hj}} = \frac{\partial E_k}{\partial \hat{y}_j^k} \cdot \frac{\partial \hat{y}_j^k}{\partial \beta_j^k} \cdot \frac{\partial \beta_j^k}{\partial w_{hj}}
$$
根据$\beta_j$定义
$$
\frac{\partial \beta_j^k}{\partial w_{hj}} = b_h
$$
鉴于sigmod函数有如下特点${f}'(x) = f(x)(1 - f(x))$，于是有
$$
{\frac{\partial E_k}{\partial \hat{y}_j^k} = (\hat{y}_j^k - y_j^k)\\
f(\beta_j -\theta_j)=\hat{y}_j^k\\
g_j = - \frac{\partial E_k}{\partial \hat{y}_j^k} \cdot \frac{\partial \hat{y}_j^k}{\partial \beta_j^k} \\
=-(\hat{y}_j^k - y_j^k){f}'(\beta_j - \theta_j)\\
=\hat{y}_j^k(1-\hat{y}_j^k)(y_j^k - \hat{y}_j^k)
}
$$
于是，可以推导出:
$$
\Delta w_hj = \eta g_j b_h
$$
类似的:
$$
{\Delta \theta_j = -\eta g_j \\
\Delta v_{ih} = \eta e_h x_i \\
\Delta \gamma_h = -\eta e_h
}
$$
其中:
$$
{e_h = -\frac{\partial E_k}{\partial b_h} \cdot \frac{\partial b_h}{\partial \alpha_k}\\
=-\sum_{j=1}^{l}-\frac{\partial E_k}{\partial \beta_j} \cdot \frac{\partial \beta_j}{\partial b_h} {f}'(\alpha_h - \gamma_h)
}
$$
步骤：

1. 根据网络计算输出结果
2. 根据输出计算误差
3. 误差前推计算各权重

## 标准BP与累计BP

$$
E = \frac{1}{m}\sum_{i=1}^{m}E_k
$$

BP算法目标是最小化训练集D上的累计误差E，标准BP是针对一个训练样例更新。还有基于累计误差推到的更新算法。区别：

标准BP根据但个样例更新，更新频繁，容易抵消。

而累计BP刷新一次BP再更新，容易达到一定进度后很难。而标准BP容易较快获得解，尤其是样本比较大的时候。


## 总结

已证明，只需要一个包含足够多神经元的隐藏层，多层前溃网络可以任意精度逼近任何复杂的连续函数，但是如何设置，依然是个未决问题，实际应用通常靠算命，试错调整。

神经网络太强大容易过拟合：

1. 早停：训练集计算梯度，更新权重等，验证集计算误差，若训练集误差下降而测试集误差上升，则停止训练
2. 正则化


## 避免局部最小

1. 以多组不同参数初始化，训练多个，取误差最小的（相当于从不同点开始搜索，降低陷入局部最有点概率）
2. 模拟退火：每一步都以一定概率接受更差结果
3. 使用随机梯度下降：加入了随机因素，使陷入局部最有解时梯度仍然不可能为零

以上没有数学理论支撑，属于***算命***



# TODO

如果解决梯度消失

CNN：权共享

普通：逐层训练


# 梯度消失与爆炸
参考REF部分

原因：链式求导中间出现>1与<1，层数增多后，出现梯度爆炸与消失的情况。

**解决方法**：选择激活函数与训练方法与网络结构：

1. Relu/elu/leakRelu
2. Batch Normalization
3. 残差结构
4. 正则化（通常L1/L2）or梯度剪切（限定在一个范围内）
5. 逐层训练（从pytorch与tensorflow的demo中已经很少看到了）



# REF

[斯坦福：UFLDL教程](http://deeplearning.stanford.edu/tutorial/)

[详解机器学习中的梯度消失、爆炸原因及其解决方法](https://blog.csdn.net/qq_25737169/article/details/78847691)